In [1]:
import numpy as np
import pandas as pd
import os
import pandas
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
pd.set_option('display.max_columns', None)

In [2]:
get_ipython().run_line_magic('pip', 'install db-dtypes')

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/opt/homebrew/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


In [3]:
get_ipython().system('pip install db-dtypes')

CONFIGURE THE BIGQUERY SETTINGS

In [4]:
BIGQUERY_PROJECT = 'ironhacks-data'
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [5]:
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data`
"""

In [6]:
# QUERY THE DATA ONCE
query_job = bigquery_client.query(query)
df = query_job.to_dataframe()

In [7]:
df.head()

,uu_id,timeperiod,week_number,countyfips,tract,tract_name,total_claims,edu_8th_or_less,edu_grades_9_11,edu_hs_grad_equiv,edu_post_hs,edu_unknown,top_category_employer1,top_category_employer2,top_category_employer3,gender_female,gender_male,gender_na,race_amerindian,race_asian,race_black,race_noanswer,race_hawaiiannative,race_other,race_white
0,f013068de98db1470bd986137a0c6d23,20220416,16,18003,900,"Census Tract 9, Allen County, Indiana",22,0,<NA>,14,<NA>,0,31-33,51,62,<NA>,<NA>,0,0,0,11,0,0,<NA>,<NA>
1,21957d5517323845818d87623589e1ba,20220319,12,18089,10400,"Census Tract 104, Lake County, Indiana",111,0,<NA>,108,<NA>,0,48-49,51,56,76,35,0,0,0,<NA>,<NA>,0,0,0
2,6a5609f385912113b6f1014b958ed748,20220326,13,18089,11500,"Census Tract 115, Lake County, Indiana",39,0,<NA>,<NA>,<NA>,0,56,N/A,N/A,<NA>,<NA>,0,0,0,39,0,0,0,0
3,46b2882ec4c373527ec33f7bd4f1388d,20220716,29,18089,20700,"Census Tract 207, Lake County, Indiana",14,0,<NA>,<NA>,<NA>,<NA>,56,51,44-45,<NA>,<NA>,0,0,<NA>,10,0,0,<NA>,<NA>
4,37495d17e82f7df326bfc2c4c090f7b7,20220409,15,18089,21900,"Census Tract 219, Lake County, Indiana",155,0,<NA>,69,83,<NA>,72,51,56,90,65,0,0,0,135,<NA>,<NA>,<NA>,<NA>


In [8]:
df.info

<bound method DataFrame.info of                                   uu_id  timeperiod  week_number  countyfips  \
0      f013068de98db1470bd986137a0c6d23    20220416           16       18003   
1      21957d5517323845818d87623589e1ba    20220319           12       18089   
2      6a5609f385912113b6f1014b958ed748    20220326           13       18089   
3      46b2882ec4c373527ec33f7bd4f1388d    20220716           29       18089   
4      37495d17e82f7df326bfc2c4c090f7b7    20220409           15       18089   
...                                 ...         ...          ...         ...   
16828  f35320206f3dd1ffc411e4ac127caf92    20220618           25       18163   
16829  f35320206f3dd1ffc411e4ac127caf92    20220827           35       18163   
16830  f35320206f3dd1ffc411e4ac127caf92    20220827           35       18163   
16831  bbcb018f0e5e49e13636f6e78ce9f60f    20220326           13       18163   
16832  bbcb018f0e5e49e13636f6e78ce9f60f    20220416           16       18163   

       

In [9]:
df.describe()

,timeperiod,week_number,countyfips,tract,total_claims,edu_8th_or_less,edu_grades_9_11,edu_hs_grad_equiv,edu_post_hs,edu_unknown,gender_female,gender_male,gender_na,race_amerindian,race_asian,race_black,race_noanswer,race_hawaiiannative,race_other,race_white
count,16833.0,16833.0,16833.0,16833.0,16833.0,13748.0,5942.0,6036.0,3246.0,12031.0,3835.0,3931.0,15868.0,15285.0,14939.0,7124.0,10089.0,16426.0,8075.0,6392.0
mean,20220540.093329,21.329531,18088.919682,211605.900315,22.952712,0.050989,1.291148,19.319583,16.923598,0.131992,18.957757,19.689138,0.005546,0.010926,0.014057,10.738069,0.207156,0.0,0.504396,17.312735
std,268.366832,11.618554,41.019467,288715.02492,14.583655,1.05481,5.320218,12.554864,10.889373,1.772384,11.939064,12.093698,0.454213,0.49316,0.568924,15.92762,2.51169,0.0,3.353956,11.623372
min,20220101.0,1.0,18001.0,100.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,20220312.0,11.0,18063.0,10100.0,14.0,0.0,0.0,12.0,11.0,0.0,12.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0
50%,20220528.0,22.0,18095.0,42801.0,18.0,0.0,0.0,15.0,14.0,0.0,15.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0
75%,20220806.0,32.0,18103.0,342600.0,27.0,0.0,0.0,22.0,20.0,0.0,22.0,23.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0,21.0
max,20220910.0,37.0,18183.0,976400.0,170.0,41.0,78.0,137.0,154.0,60.0,126.0,139.0,49.0,36.0,41.0,163.0,110.0,0.0,74.0,155.0
